In [2]:
## MAIN WINE QUALITY FRAME

import pandas as pd, numpy as np

## Import raw data
df_white = pd.read_csv('raw/winequality-white.csv', sep = ';')
df_red = pd.read_csv('raw/winequality-red.csv', sep = ';')

## Removing outliers
def remove_outliers(df, columns, qtile):
    def df_cutter(df, column, qtile):
        limit = df.loc[:, column].quantile(qtile)
        return df.loc[df[column] < limit, ]
    for col in columns:
        name = df.columns[col]
        df = df_cutter(df, name, qtile)
    return df

df_white = remove_outliers(df_white, [2,4,5,6, 7], 0.999)
df_red = remove_outliers(df_red, [3,4,6,9, 7], 0.999)

df_white['wine'] = 'white'
df_red['wine'] = 'red'

## Merging dataframes
wine_quality = pd.concat([df_red, df_white])

## Adding relative quality column
wine_quality['quality factor'] = wine_quality['quality'].apply(lambda x: 'Below Average' if x < 6 else ('Above Average' if x > 6 else 'Average'))

## Adding numeric version of the realtive quality columns
wine_quality['quality factor numeric'] = wine_quality['quality factor'].apply(lambda x: 0 if x=='Below Average' else (1 if x=='Average' else 2))


## Reorder Columns so Wine [-3] comes first
cols = wine_quality.columns.to_list()
cols = [cols[-3]] + cols[:-3] + cols[-2:]
wine_quality = wine_quality[cols]

## Capitalize columns
wine_quality.columns = map(str.title, wine_quality.columns)
wine_quality = wine_quality.rename(columns={'Ph': 'pH'})

## Export
wine_quality.to_csv('processed_python/wine_quality.csv', index=False)

## Test import
imported_df = pd.read_csv('processed_python/wine_quality.csv')

imported_df.loc[imported_df.Wine == 'white', ].describe()


,Fixed Acidity,Volatile Acidity,Citric Acid,Residual Sugar,Chlorides,Free Sulfur Dioxide,Total Sulfur Dioxide,Density,pH,Sulphates,Alcohol,Quality,Quality Factor Numeric
count,4871.000000,4871.000000,4871.000000,4871.000000,4871.000000,4871.000000,4871.000000,4871.000000,4871.000000,4871.000000,4871.000000,4871.000000,4871.000000
mean,6.851375,0.277631,0.333082,6.347208,0.045489,35.152433,137.908951,0.993998,3.188292,0.489752,10.515639,5.881749,0.883802
std,0.842965,0.099986,0.117052,4.946691,0.020322,16.362965,41.684581,0.002893,0.150974,0.114176,1.230599,0.883558,0.733509
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000,0.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991715,3.090000,0.410000,9.500000,5.000000,0.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993700,3.180000,0.470000,10.400000,6.000000,1.000000
75%,7.300000,0.320000,0.390000,9.800000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000,1.000000
max,14.200000,1.100000,0.990000,23.500000,0.244000,124.000000,260.000000,1.002400,3.820000,1.080000,14.200000,9.000000,2.000000


In [5]:
## CORRELATION PLOT DATA EXPORT

wine = wine_quality #This is the naming convention I had in my work and didn't want to change it

# Get correlations for each wine type
corr_df_white = wine.loc[wine['Wine'] == 'white'].select_dtypes('number').corr('spearman').stack().reset_index(name='corr')
corr_df_white["Wine"] = "white"

corr_df_red = wine.loc[wine['Wine'] == 'red'].select_dtypes('number').corr('spearman').stack().reset_index(name='corr')
corr_df_red["Wine"] = "red"

# Bind them together
corr_df = corr_df_white.append(corr_df_red)
corr_df["Quality Factor"] = 3 # For all qualities

# Subset by quality and for each and bind
for i in [0,1,2]:
    #Create white df at ith quality
    corr_df_white = wine.loc[(wine['Wine'] == 'white') & (wine["Quality Factor"] == i)].select_dtypes('number').corr('spearman').stack().reset_index(name='corr')
    corr_df_white["Wine"] = "white"
    corr_df_white["Quality Factor"] = i
    #create red df at ith quality
    corr_df_red = wine.loc[(wine['Wine'] == 'red') & (wine["Quality Factor"] == i)].select_dtypes('number').corr('spearman').stack().reset_index(name='corr')
    corr_df_red["Wine"] = "red"
    corr_df_red["Quality Factor"] = i
    # bind to main df
    corr_df = corr_df.append(corr_df_red)
    corr_df = corr_df.append(corr_df_white)


corr_df.to_csv('processed_python/correlation.csv', index=False)

## Test import
pd.read_csv('processed_python/correlation.csv').head()


,level_0,level_1,corr,Wine,Quality Factor
0,Fixed Acidity,Fixed Acidity,1.000000,white,3
1,Fixed Acidity,Volatile Acidity,-0.047202,white,3
2,Fixed Acidity,Citric Acid,0.290753,white,3
3,Fixed Acidity,Residual Sugar,0.084902,white,3
4,Fixed Acidity,Chlorides,0.091888,white,3
